In [ ]:
using Pkg
Pkg.add("Match")
Pkg.add("DataFrames")
Pkg.add("CSV")

using Distributed
using Match
using DataFrames
using CSV

In [ ]:
guesses = readlines("valid_guesses.txt")
solutions = readlines("valid_solutions.txt")

In [ ]:
uints(s::String)::Vector{UInt8} = [UInt8(c) - 96 for c in s]

_guesses = [uints(s) for s in guesses]
_solutions = [uints(s) for s in solutions]

In [ ]:
function colors(solution::Vector{UInt8}, guess::Vector{UInt8})::UInt8
    colors_ = 0
    remaining = zeros(UInt8, 26)
    unchecked = UInt8[]

    # greens
    for (i, c) in enumerate(solution)
        if c == guess[i]
            colors_ += 2 * 3^(i-1)
        else
            push!(unchecked, i)
            remaining[c] += 1
        end
    end
    
    # yellows
    for i in unchecked
        if remaining[guess[i]] > 0
            colors_ += 3^(i-1)
            remaining[guess[i]] -= 1
        end
    end
    colors_
end

time_colors() = @time colors(_solutions[1], _solutions[1])
time_colors()

In [ ]:
function colors(solutions::Vector{Vector{UInt8}}, guess::Vector{UInt8})::Vector{UInt8}
    colors_ = UInt8[]
    for s in solutions
        push!(colors_, colors(s, guess))
    end
    colors_
end

time_colors() = @time colors(_solutions, _guesses[1])
time_colors()

In [ ]:
function entropy(colors::Vector{UInt8})::Float64
    freqs = zeros(UInt16, 243)
    for i in colors
        freqs[i+1] += 1
    end
    
    dist = filter(!iszero, freqs ./ sum(freqs))
    -sum(dist .* log2.(dist))
end

const trial_1 = colors(_solutions, _guesses[1])

time_entropy() = @time entropy(trial_1)
time_entropy()

In [ ]:
function bestguess(solutions::Vector{Vector{UInt8}}, guesses::Vector{Vector{UInt8}})::Vector{UInt8}
    entropies = Float64[]
    for s in guesses
        push!(entropies, entropy(colors(solutions, s)))
    end
    _, i = findmax(entropies)
    guesses[i]
end

time_bestguess() = @time bestguess(_solutions, _solutions)
time_bestguess()
    

In [ ]:
string(v::Vector{UInt8})::String = String([Char(c+96) for c in v])
println(string(bestguess(_solutions, _guesses)))
println(string(bestguess(_solutions, _solutions)))

In [ ]:
function matches(solutions::Vector{Vector{UInt8}}, guess::Vector{UInt8}, response::UInt8)::Vector{Vector{UInt8}}
    matches_ = Vector{UInt8}[]
    responses = colors(solutions, guess)
    
    for (i, r) in enumerate(responses)
        if response == r
            push!(matches_, solutions[i])
        end
    end
    matches_
end

time_matches() = @time matches(_solutions, _guesses[1], colors(_solutions[1], _guesses[1]))
time_matches()

In [ ]:
function string(r::UInt8)::String
    s_ = Char[]
    for i in 1:5
        @match r % 3 begin
            0 => push!(s_, 'n')
            1 => push!(s_, 'y')
            2 => push!(s_, 'g')
        end
        r ÷= 3
    end
    String(s_)
end

string(trial_1[1])

In [ ]:
function play(solutions::Vector{Vector{UInt8}}, guesses::Vector{Vector{UInt8}}, solution::Vector{UInt8})::Vector{String}
    sequence_ = fill("", 12)
    
    first = [0x13, 0x0f, 0x01, 0x12, 0x05] # soare
    response = colors(solution, first)
    
    sequence_[1] = "soare"
    sequence_[7] = string(response)
    
    remaining = matches(solutions, first, response)
    
    for i in 2:6
        if size(remaining) == 1
            sequence_[i] = string(remaining[1])
            sequence_[i+6] = string(242)
            return sequence_
        end
        
        guess = bestguess(remaining, remaining)
        responses = colors(remaining, guess)
        
        h1 = entropy(responses)
        hg = -log2(1/size(remaining)[1])
        
        if h1 != hg
            reduce = bestguess(remaining, guesses)
            responses = colors(remaining, reduce)
            
            h2 = entropy(responses)
            if h2 - h1 > 0.2
                guess = reduce
            end
        end
        
        response = colors(solution, guess)
        
        sequence_[i] = string(guess)
        sequence_[i+6] = string(response)
        
        if response == 242 # correct
            return sequence_
        end
        
        remaining = matches(remaining, guess, response)
    end
    sequence_
end

time_play() = @time play(_solutions, _guesses, _solutions[1])
time_play()

In [ ]:
function play(solutions::Vector{Vector{UInt8}}, solution::Vector{UInt8})::Vector{String}
    sequence_ = fill("", 12)
    
    first = [0x13, 0x0f, 0x01, 0x12, 0x05] # soare
    response = colors(solution, first)
    
    sequence_[1] = "soare"
    sequence_[7] = string(response)
    
    remaining = matches(solutions, first, response)
    
    for i in 2:6
        if size(remaining) == 1
            sequence_[i] = string(remaining[1])
            sequence_[i+6] = string(242)
            return sequence_
        end
        
        guess = bestguess(remaining, remaining)
        responses = colors(remaining, guess)
        
        response = colors(solution, guess)
        
        sequence_[i] = string(guess)
        sequence_[i+6] = string(response)
        
        if response == 242 # correct
            return sequence_
        end
        
        remaining = matches(remaining, guess, response)
    end
    sequence_
end

time_play() = @time play(_solutions, _solutions[1])
time_play()

In [ ]:
function solveall()
    solved = DataFrame(
        g1 = String[],
        g2 = String[],
        g3 = String[],
        g4 = String[],
        g5 = String[],
        g6 = String[],
        r1 = String[],
        r2 = String[],
        r3 = String[],
        r4 = String[],
        r5 = String[],
        r6 = String[]
    )
    
    Threads.@threads for s in _solutions
        push!(solved, play(_solutions, _guesses, s))
    end
    
    CSV.write("solved.csv", solved)
end

time_solveall() = @time solveall()
time_solveall()

In [ ]:
function solveall2()
    solved = DataFrame(
        g1 = String[],
        g2 = String[],
        g3 = String[],
        g4 = String[],
        g5 = String[],
        g6 = String[],
        r1 = String[],
        r2 = String[],
        r3 = String[],
        r4 = String[],
        r5 = String[],
        r6 = String[]
    )
    
    Threads.@threads for s in _solutions
        push!(solved, play(_solutions, s))
    end
    
    CSV.write("solved_2.csv", solved)
end

time_solveall() = @time solveall2()
time_solveall()

In [ ]:
const solved = DataFrame(CSV.File("solved.csv"))
const solves = @view solved[!, 1:6]

const solved2 = DataFrame(CSV.File("solved_2.csv"))
const solves2 = @view solved2[!, 1:6]

In [ ]:
tries = [count(!ismissing, r) for r in eachrow(solves)]
println(sum(tries) / length(tries))

tries = [count(!ismissing, r) for r in eachrow(solves2)]
println(sum(tries) / length(tries))

In [ ]:
value_counts(df, col) = combine(groupby(df, col), nrow)
sort(value_counts(solves, :g2), :nrow, rev=true)

In [ ]:
clint_2 = filter(:g2 => ==("clint"), solved)
sort(value_counts(clint_2, :r1), :nrow, rev=true)